In [ ]:
# Importing required libraries for optimization
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [ ]:
# Least squares solution for Ax = b
def least_squares(A, b):
    # A: matrix of coefficients
    # b: vector of constants

    return np.linalg.inv(A.T @ A) @ A.T @ b

In [ ]:
# Finite difference methods:

# Forward difference:
def forward_diff(f, x, h=1E-08):
    # f: function
    # x: vector of variables
    # h: step size

    delta_f = np.zeros(x.shape) # Initialize delta_f
    for i in range(x.shape[0]):
        x_forw = np.array(x) # Make a copy of x
        x_forw[i] += h   # Increment x_forw[i] by h
        delta_f[i] = (f(x_forw) - f(x)) / h # Calculate the forward difference

    return delta_f


# Central difference:
def central_diff(f, x, h=1E-08):
    # f: function
    # x: vector of variables
    # h: step size

    delta_f = np.zeros(x.shape) # Initialize delta_f
    for i in range(x.shape[0]):
        x_forw = np.array(x) # Make a copy of x
        x_back = np.array(x) # Make a copy of x
        x_forw[i] += h   # Increment x_forw[i] by h
        x_back[i] -= h   # Decrement x_back[i] by h
        delta_f = (f(x_forw) - f(x_back)) / (2*h) # Calculate the central difference

    return delta_f

In [ ]:
# Converting constrained optimization problem to unconstrained optimization problem:
# Using penalty method:
def constrained_to_unconstrained(f, g, h, x, p=100):
    # f: objective function
    # g: inequality constraint
    # h: equality constraint
    # x: vector of variables
    # p: penalty parameter

    # Objective function
    def f_unconstrained(x):
        return f(x) + p*(g(x)**2 + h(x)**2)

    return f_unconstrained

In [ ]:
# Unconstrained optimization:
# 1st order methods:

# Steepest descent method:
def steepest_descent(f, x, delta_f, delta_f_tol=1E-08, delta_x_tol=1E-08, max_iter=1000, full_output=False):
    # f: function
    # x: vector of variables
    # delta_f: method for calculating the gradient

    # exit flags:
    # 0: maximum number of iterations reached
    # 1: gradient is within tolerance
    # 2: design variable update is within tolerance

    convergence = False
    iter = 0
    while not convergence:
        s = -delta_f(f, x) # Calculate the search direction
        s = s / np.sqrt(np.dot(s, s)) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the step size
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter
        
        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.sqrt(np.dot(delta_f(f, x_new), delta_f(f, x_new))) <= delta_f_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.sqrt(np.dot(x_new - x, x_new - x)) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        x = x_new # Update x

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new


# Conjugate gradient method:
def conjugate_gradient(f, x, delta_f, delta_f_tol=1E-08, delta_x_tol=1E-08, max_iter=1000, full_output=False):
    # f: function
    # x: vector of variables
    # delta_f: method for calculating the gradient
    
    # exit flags:
    # 0: maximum number of iterations reached
    # 1: gradient is within tolerance
    # 2: design variable update is within tolerance

    s = -delta_f(f, x) # Calculate the initial search direction
    s = s / np.sqrt(np.dot(s, s)) # Normalize the initial search direction
    alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the initial step size
    x_new = x + alpha*s # Calculate the initial x update

    convergence = False
    iter = 0
    while not convergence:
        if iter % x.shape[0] == 0:  # Check if the iteration counter is a multiple of the number of design variables
            s = -delta_f(f, x_new) # Calculate the search direction
        else:
            s = -delta_f(f, x_new) + np.sqrt(np.dot(delta_f(f, x_new), delta_f(f, x_new)) / np.dot(delta_f(f, x), delta_f(f, x))) * s # Calculate the search direction

        s = s / np.sqrt(np.dot(s, s)) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x_new + alpha*s), 0, 1) # Calculate the step size
        x = x_new # Update x
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter

        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.sqrt(np.dot(delta_f(f, x_new), delta_f(f, x_new))) <= delta_f_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.sqrt(np.dot(x_new - x, x_new - x)) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        
    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new

# Quasi-Newton methods:
# BFGS update rule:
def BFGS_update(B, delta_x, delta_delta):
    # B: Hessian approximation
    # delta_x: design variable update
    # delta_delta: gradient update

    return (1 + delta_delta.T@B@delta_delta / (delta_delta.T@delta_x)) * (delta_x@delta_x.T) / (delta_x.T@delta_delta) - \
        (delta_x@(delta_delta.T@B) + (delta_delta.T@B).T@delta_x.T) / (delta_x.T@delta_delta)

# DFP update rule:
def DFP_update(B, delta_x, delta_delta):
    # B: Hessian approximation
    # delta_x: design variable update
    # delta_delta: gradient update

    return delta_x@delta_x.T / (delta_x.T@delta_delta) - \
        (B@delta_delta)@(B@delta_delta).T / (delta_delta.T@B@delta_delta)

# Quasi-Newton method:
def quasi_newton(f, x, delta_f, delta_f_tol=1E-08, delta_x_tol=1E-08, max_iter=1000, full_output=False, update_rule=BFGS_update):
    # f: function
    # x: vector of variables
    # delta_f: method for calculating the gradient
    # update_rule: method for updating the Hessian approximation

    # exit flags:
    # 0: maximum number of iterations reached
    # 1: gradient is within tolerance
    # 2: design variable update is within tolerance

    B = np.eye(x.shape[0]) # Initialize the Hessian approximation
    convergence = False
    iter = 0

    while not convergence:
        s = -B@delta_f(f, x)   # Calculate the search direction
        s = s / np.sqrt(np.dot(s, s)) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the step size
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter

        # Check for convergence:
        if iter >= max_iter:
            convergence = True
            exit_flag = 0
        elif np.sqrt(np.dot(delta_f(f, x_new), delta_f(f, x_new))) <= delta_f_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.sqrt(np.dot(x_new - x, x_new - x)) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2

        B = update_rule(B, x_new - x, delta_f(f, x_new) - delta_f(f, x)) # Update the Hessian approximation
        x = x_new # Update x

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new

In [ ]:
# Constrained optimization methods:
# Function to calculate the lagrangian of function:
def calc_lagrangian(f, con, x, lambda_, mu):
    # f: function to be minimized
    # x: vector of variables
    # lambda_: vector of lagrange multipliers for inequality constraints
    # mu: vector of lagrange multipliers for equality constraints
    # g: vector of inequality constraints
    # h: vector of equality constraints

    
    g, h = con(x)   # Calculate the inequality and equality constraints
    
    return f(x) + mu@g + lambda_@h  # Calculate the lagrangian


# Augmented Lagrangian method:
# Function to calculate the augmented Lagrangian function:
def calc_augmented_lagrangian(f, con, x, lambda_, mu, rho):
    # f: objective function
    # con: constraint function
    # x: vector of variables
    # lambda_: Lagrange inequality multiplier
    # mu: Lagrange equality multiplier
    # rho: penalty parameter

    g, h = con(x) # Calculate the inequality and equality constraints
    
    return calc_lagrangian(f, con, x, lambda_, mu) + rho*(np.dot(g, g) + np.dot(h, h)) # Calculate the augmented Lagrangian function

def augmented_lagrangian(f, con, x, delta_f, rho=1, delta_f_tol=1E-08, delta_x_tol=1E-08, max_iter=1000, full_output=False):
    # f: function
    # x: vector of variables
    # con: constraint function
    # delta_f: method for calculating the gradient
    
    # exit flags:
    # 0: maximum number of iterations reached
    # 1: lagrangian gradient is within tolerance
    # 2: design variable update is within tolerance
    
    g, h = con(x) # Calculate the initial constraint values
    mu0 = np.zeros(g.shape[0]) # Initialize the inequality Lagrangian multipliers
    lambda_0 = np.zeros(h.shape[0]) # Initialize the equality Lagrangian multipliers
    convergence = False
    iter = 0

    while not convergence:
        x_new = quasi_newton(lambda x: calc_augmented_lagrangian(f, con, x, lambda_0, mu0, rho), \
            x, delta_f, delta_f_tol, delta_x_tol, max_iter, full_output=False) # Optimize the augmented Lagrangian function
        g, h = con(x_new) # Calculate the constraint values
        mu_new = mu0 + rho*g # Calculate the new inequality Lagrangian multipliers
        # Check for negative inequality multipliers:
        for i in range(len(mu_new)):
            if mu_new[i] < 0:
                mu_new[i] = 0
        lambda_new = lambda_0 + rho*h # Calculate the new equality Lagrangian multipliers
        delta_L_new = delta_f(lambda x: calc_lagrangian(f, con, x, lambda_new, mu_new), x_new) # Calculate the new Lagrangian gradient
        iter += 1 # Increment the iteration counter
        
        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.sqrt(np.dot(delta_L_new, delta_L_new)) <= delta_f_tol: # Check if lagrangian gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.sqrt(np.dot(x_new - x, x_new - x)) <= delta_x_tol: # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        
        x = x_new # Update x
        mu0 = mu_new # Update the Lagrangian multipliers
        lambda_0 = lambda_new # Update the Lagrangian multipliers

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new